In [92]:
import os
import shutil

def copy_image(image_path, target_path):
    shutil.copy(image_path, target_path)

output_dir = './output/'
input_dir = './image_input/'
try:
    os.makedirs('./output')
except:
    pass

for image in os.listdir(input_dir):
    image = "".join(image.split('.')[:-1:])

    try:
        os.makedirs(output_dir+image)
    except:
        pass
    try:
        copy_image(input_dir+image+'.png', output_dir+image+'/')
    except:
        pass
    

# EXTRACT SIDEBAR

In [93]:
from PIL import Image, ImageDraw
import os
import shutil
import json 


current_path = os.getcwd() 

output_path = "./output/"



for name in os.listdir(output_path):
    # only take the sidebar
    current_image_path = output_path+'/'+name+'/'
    output_dir = current_image_path + "/sidebar/"
    input_image_path = current_image_path + name + '.png'
    image = Image.open(input_image_path)
    
    # 1/7 is the sidebar part 
    image = image.crop((0,0,70,image.height)) #(1/6)
    
    try:
        os.makedirs(output_dir)
    except:
        pass
    
    print(f'Sidebar ... {output_dir+name}')

    image.save(output_dir+name+'.png')
    print(f'Sidebar ... done')


print("Done")

Sidebar ... ./output//input//sidebar/input
Sidebar ... done
Done


# EXTRACT SECTIONS

In [94]:
import json 
from PIL import Image, ImageDraw, ImageFont
import os 
import numpy as np
import cv2

def overlay_squares_with_labels(original_img_path, squares, output_path):
    """
    Overlays the detected black squares with a red box and labels the height of each square.
    
    Args:
    original_img_path (str): Path to the original image.
    squares (list of tuples): Each tuple contains (start_height, height) for each black square.
    output_path (str): Path to store the result image.
    """
    # Load the original image
    img = Image.open(original_img_path)
    draw = ImageDraw.Draw(img)
    
    # Define the box color and text color
    box_color = (255, 0, 0)  # Red color
    text_color = (255, 0, 0) # Red text
    
    # Load a font for the height labels
    # Using default font as PIL may not always have a specific font path on the system
    try:
        font = ImageFont.truetype("arial.ttf", 15)
    except IOError:
        font = ImageFont.load_default()

    # Get image width (to make the box span across the image width)
    img_width, img_height = img.size

    # Overlay each square with a red box and label it with its height
    for start, height,x_min,x_max in squares:
        # Draw a red rectangle around each black square
        draw.rectangle([x_min, start, x_max, start + height], outline=box_color, width=2)
        
        # Add the height label next to the square
        draw.text((img_width - 50, start - 15), f"{start}px", fill=text_color, font=font)
    
    # Save the resulting image
    img.save(output_path)
    # print(f"Image saved at {output_path}")

# Example usage with the provided squares and a new output path

def get_black_square_data(image,  min_size=15):
    # Convert the image to grayscale to help detect the black squares
    gray_img = image.convert('L')
    gray_img.point(lambda x: 0 if x < 128 else 255, '1')
    
    # Convert the PIL image to a NumPy array
    binary_image = np.array(gray_img.copy())

    # Threshold the array to ensure it's binary
    binary_image = (binary_image < 128).astype(int)  # Assuming black is below 128
    
    # Ensure the binary image is in the correct format
    if binary_image.dtype != np.uint8:
        binary_image = binary_image.astype(np.uint8)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_image =  image.copy()

    # List to store rectangle properties
    rectangles = []

    # Iterate over contours
    for contour in contours:
        # Get the bounding box for each contour
        x, y, w, h = cv2.boundingRect(contour)
        
        # Check if the bounding box is a square and larger than 15x15
        if w >= min_size and h >= min_size and abs(w - h) <= 2:  # Allow a small tolerance for non-perfect squares
            # Append the rectangle properties: (start_height, height, x_min, x_max)
            rectangles.append((y, h, x, x + w))
            
            draw = ImageDraw.Draw(contour_image)
            contour_points = [(int(point[0][0]), int(point[0][1])) for point in contour]
            draw.polygon(contour_points, outline=(0, 255, 0), width=2)
            # contour_image = draw.polygon(contour, fill=(0,255,0))

    
    
    return rectangles, gray_img, contour_image


output_path = "./output/"

for image_dir in os.listdir(output_path):
    output_dir = output_path+image_dir + '/square_recognition/'
    try:
        os.makedirs(output_dir)
    except:
        pass
    
    
    # Load the image
    input_image_path = output_path+image_dir + '/sidebar/'+ image_dir + '.png'

    img = Image.open(input_image_path)


    # Get the heights and start positions of the black squares
    black_square_info, gray_img, contour_image = get_black_square_data(img)
    gray_img.save(output_dir+image_dir+'_bw.png')
    contour_image.save(output_dir+image_dir+'_countors.png')
    
    # filter too high
    black_square_info = [x for x in black_square_info if x[3] - x[2] > 10]

    # sort
    black_square_info.sort(key=lambda x: x[0])

    with open(output_dir+image_dir+'.json', 'w') as f:
        json.dump(black_square_info, f)
    
    overlay_squares_with_labels(input_image_path, black_square_info, output_dir+image_dir+'.png')



# SECTION EXTRACTION

In [95]:
from PIL import Image, ImageDraw
import os
import shutil
import json 


def clamp(n, min, max): 
    if n < min: 
        return min
    elif n > max: 
        return max
    else: 
        return n 


output_path = "./output/"




for name in os.listdir(output_path):
    image_path = output_path+name+'/'+name+'.png'
    
    image = Image.open(image_path)
    

    with open(output_path+name+'/square_recognition/'+name+'.json', 'r') as f:
        square_data = json.load(f)    
    
    square_heights = [square[0] for square in square_data]
    square_x_max = [square[3] for square in square_data]
    square_heights.append(image.height)

    for index, h_break in enumerate(square_heights):
        
        if (index >= len(square_heights) - 1 ):
            continue
        
        y = clamp(h_break - 7, 0, image.height - 1)
        next_y = clamp(square_heights[index+1] - 5, 0, image.height - 1)
        
        # min height and fix
        if (next_y < y or next_y - y < 30):
            continue
        
        # kantlijn, boven, einde van pagina, beneden grens
        crop = (0, y, image.width, next_y)
        
        section_image = image.copy()
        
        cropped = section_image.crop(crop)        

        # section_image.show()
        # section_file_name = filenameify(h_break["description"])+".png"
        
        output_dir = output_path + name + '/sections/' + str(index) + '/'
        try:
            os.makedirs(output_dir)
        except:
            pass
        
        section_name = name+"_s"+str(index)
        cropped.save(output_dir+section_name+'.png')

        
        
        try:
            os.makedirs(output_dir+'question_selection/')
        except:
            pass
        
        question_selection_crop = (square_x_max[index], 0, int(cropped.width * (1/8)), cropped.height)
        question_selection_image = cropped.copy().crop(question_selection_crop)
        question_selection_image.save(output_dir+'question_selection/'+section_name+'.png')
        
        try:
            os.makedirs(output_dir+'answer/')
        except:
            pass
        
        answer_crop = (int(cropped.width * (1/8) ), 0, cropped.width, cropped.height)
        answer_image = cropped.copy().crop(answer_crop)

        answer_image.save(output_dir+'answer/'+section_name+'.png')


In [96]:
import os
import json


from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

output_path = "./output/"


with open("creds/microsoft.json", "r") as f:
    data = json.load(f)
    endpoint = data["endpoint"]
    credential = AzureKeyCredential(data["subscription_key"])
    document_intelligence_client = DocumentAnalysisClient(endpoint, credential)

for name in os.listdir(output_path):
    
    for section in os.listdir(output_path+name+'/sections/'):
        section_dir = output_path+name+'/sections/'+section+'/'
        section_name = name+'_s'+section
        input_dir = section_dir+'question_selection/'+section_name+'.png'
        
        output_dir = section_dir + 'question_recognition/'
        try:
            os.makedirs(output_dir)
        except:
            pass
                
        print('Starting ' + image_dir)
        # sample document
        
        with open(input_dir, 'rb') as f:
        
            poller = document_intelligence_client.begin_analyze_document(model_id="prebuilt-document", document=f)
        
        result = poller.result().to_dict()
        
        with open(output_dir+section_name+'.json', 'w') as f:
            json.dump(result, f, indent=4)
        print('Finished ' + image_dir)

print('Done')

Starting input
Finished input
Starting input
Finished input
Starting input
Finished input
Starting input
Finished input
Starting input
Finished input
Starting input
Finished input
Starting input
Finished input
Done


In [106]:
import os
import json 
from PIL import Image
import re


def stack_images_vertically(img1, img2):

    # Get the dimensions of both images
    width1, height1 = img1.size
    width2, height2 = img2.size

    # Calculate the dimensions for the new image
    new_width = max(width1, width2)
    new_height = height1 + height2

    # Create a new blank image with the calculated dimensions
    new_image = Image.new('RGB', (new_width, new_height))

    # Paste the first image at the top
    new_image.paste(img1, (0, 0))

    # Paste the second image below the first image
    new_image.paste(img2, (0, height1))
    
    return new_image

#match left and right single quotes
single_quote_expr = re.compile(r'[\u2018\u2019]', re.U)
#match all non-basic latin unicode
unicode_chars_expr = re.compile(r'[\u0080-\uffff]', re.U)

def cleanse_unicode(s):
    if not s:
        return ""
    
    temp = single_quote_expr.sub("'", s, re.U)
    temp = unicode_chars_expr.sub("", temp, re.U)
    return temp

def filenameify(s):
    s = cleanse_unicode(s)
    replace_dict = {
        "①": "1",
        "②": "2",
        "③": "3",
        "④": "4",
        "⑤": "5",
        "⑥": "6",
        "⑦": "7",
        "⑧": "8",
        "⑨": "9",
        "⑩": "10",
        "⑪": "11",
        "⑫": "12",
        "⑬": "13",
        "⑭": "14",
        "⑮": "15",
        "⑯": "16",
        "⑰": "17",
        "⑱": "18",
        "⑲": "19",
        "⑳": "20",
        ".": "",
        "\n": "",
        # "/": ""
    }
    
    # https://www.geeksforgeeks.org/python-replace-words-from-dictionary/
    replaced_s = " ".join(replace_dict.get(ele, ele) for ele in s.split())
    
    filename = re.sub(r'[^a-zA-Z0-9._-]', '', replaced_s)
    
    if (len(filename) == 0):
        return "NOT_VALID_QUESTION_ID"
    
    return filename
for name in os.listdir(output_path):
    
    section_path = output_path+name+'/sections/'
    
    section_data = []
    
    for section in os.listdir(output_path+name+'/sections/'):
        section_dir = section_path +section+'/'
        section_name = name+'_s'+section
        
        with open(section_dir+'question_recognition/'+section_name+'.json', 'r') as f:
            question_data = json.load(f)
        
        data = []
        
        for item in question_data["key_value_pairs"]:
            if (item["value"]):
                if (item["value"]["content"] == ":selected:"):
                    data.append((filenameify(item["key"]["content"]), item["confidence"]))
            
        # highest confidence
        data.sort(key=lambda a: a[1], reverse=True)
        
        if (len(data) > 0):
            most_confident_item = data[0]
            section_data.append({
                "section_index": most_confident_item[0].split('.')[0],
                "question_number": most_confident_item[0].split('.')[1],
                "path_index": section,
                "confidence": most_confident_item[1]
            })
    
    
    
    question_number_dict = {}
    
    for section_info in section_data:
        if (section_info["question_number"] not in question_number_dict):
            question_number_dict[section_info["question_number"]] = []
        
        question_number_dict[section_info["question_number"]].append(section_info)
    
    
    question_output = output_path + name + '/questions/'
    
    try:
        os.makedirs(question_output)
    except:
        pass



    for question_id in question_number_dict.keys():
        images = sorted(question_number_dict[question_id], key=lambda x: x["section_index"])
        if len(images) > 0:
            
            
            stacked_image_path = section_path+images[0]["path_index"]+'/answer/'+name+'_s'+images[0]["path_index"]+'.png'
            stacked_image = Image.open(stacked_image_path)
            for section in images[1::]:
                new_image_path = section_path+section["path_index"]+'/answer/'+name+'_s'+section["path_index"]+'.png'
                new_image = Image.open(new_image_path)

                
                stacked_image = stack_images_vertically(stacked_image, new_image)
            
            question_id_path = question_output+question_id+'/'
            try:
                os.makedirs(question_id_path)
            except:
                pass

            stacked_image.save(question_id_path+name+'_q'+question_id+'.png')
            
            
    

('1.1', 0.995)
('2.1', 0.995)
('3.4', 0.995)
('4.1', 0.995)
[   {   'confidence': 0.995,
        'path_index': '0',
        'question_number': '1',
        'section_index': '1'},
    {   'confidence': 0.995,
        'path_index': '1',
        'question_number': '1',
        'section_index': '2'},
    {   'confidence': 0.995,
        'path_index': '2',
        'question_number': '4',
        'section_index': '3'},
    {   'confidence': 0.995,
        'path_index': '3',
        'question_number': '1',
        'section_index': '4'}]


In [115]:
from openai import OpenAI
import base64
import tiktoken
import json
from PIL import Image
import math
from pydantic import BaseModel
from io import BytesIO
import os
import shutil
import time
import uuid

current_path = os.getcwd() 

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Returns the number of tokens used by a list of messages."""
    #   https://github.com/openai/tiktoken?tab=readme-ov-file#-tiktoken
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
        
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for data_value in message["content"]:
            if (data_value["type"] == "text"):
                data = data_value["text"]
            
            if (data_value["type"] == "image_url"):
                data = data_value["image_url"]["url"]    
            if (data):
                num_tokens += len(encoding.encode(data))
            # if data_value["type"] == "name":  # if there's a name, the role is omitted
            #     num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

# https://platform.openai.com/docs/guides/vision 
def encode_image(image_path):
    pillow_image = Image.open(image_path)
    
    percent = 1

    new_width = int(pillow_image.width * percent)
    new_height = int(float(pillow_image.size[1]) * percent)
    
    resized_image = pillow_image.resize((new_width, new_height))
    buffered = BytesIO()
    resized_image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def copy_image(image_path, target_path):
    shutil.copy(image_path, target_path)


# schema classes
class SpellingCorrection(BaseModel):
    original: str
    changes: str

# answer class schema
class QuestionAnswer(BaseModel):
    # let openai reasses the question number (they are better than google )
    certainty: float 
    student_handwriting_percent: float
    # get the unchanged raw tekst
    raw_text: str
    # get the spel corrected tekst that should be graded
    correctly_spelled_text: str
    # get the spelling changes the model made
    spelling_corrections: list[SpellingCorrection]

def get_client():
    with open("creds/openaikey.json", "r") as f:
        openai_key = json.load(f)["key"]
    

    openai_client = OpenAI(api_key=openai_key)
    
    return openai_client

def get_response_json(response, gpt_model, start_time, end_time):
    
    dict_response = response.to_dict()
    choice_response = dict_response["choices"][0]
    result_data = choice_response["message"]["parsed"]
    request_data = dict_response["usage"]

    output_json = {

        "question_number": result_data["question_number"],
        "raw_text": result_data["raw_text"],
        "correctly_spelled_text": result_data["correctly_spelled_text"],
        "spelling_corrections": result_data["spelling_corrections"],
        
        "tokens_used": request_data["total_tokens"],

        "model_used": gpt_model,
        "model_version": dict_response["model"],
        
        "timestamp": int(end_time),
        "delta_time_s": end_time - start_time,
    }



    
    return output_json


def single_request(messages, name, question):
    
    openai_client = get_client()
    
    gpt_model = "gpt-4o-mini"

    start_time = time.time()

    print("GPT request... ")
    
    try:
        response = openai_client.beta.chat.completions.parse(
            model=gpt_model,
            temperature=0.05,
            messages=messages,
            response_format=QuestionAnswer,
            # max_tokens=30_000, #test image was 15k tokens
            timeout=14
        )
    except:
        print("GPT request... ERROR")
        return False
    print("GPT request... Done")

    end_time = time.time()

    # from pprint import pprint
    # pprint(response)
    json_output_path = f"./output/{name}/questions/{question}/json/"


    try:
        os.makedirs(json_output_path)
    except:
        pass

    # copy image here
    # copy_image(input_image_path, image_json_output_path)
        

    json_output_path = json_output_path + "/" + gpt_model + ".json"

    reponse_json = get_response_json(response, gpt_model, start_time, end_time)
    reponse_json["question_id"] = question
    json_string = json.dumps(reponse_json, indent=4)

    with open(json_output_path, "w") as outfile:
        outfile.write(json_string)
        
    print("Written to file "+json_output_path)

def batch_request(api_requests):
    openai_client = get_client()
    
    gpt_model = "gpt-4o-mini"

    request_id = uuid.uuid4()

    start_time = time.time()

    print("GPT request... ")
    request_data = []
    
    for (i, api_request) in enumerate(api_requests):
        request_data.append({
            "custom_id": "request-"+i, 
            "method": "POST", 
            "url": "/v1/chat/completions", 
            "body": api_request
        })
    
    
    with open("batchinput.jsonl", "w") as f:
        f.write(request_data)
    
    input_file = openai_client.files.create(
        file=open("batchinput.jsonl", "rb"),
        purpose="batch"
    )
    
    try:
        batch_input_file_id = input_file.id

        batch_data = openai_client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="h",
            metadata={
                "description": "nightly eval job"
            }
        )
    except:
        print("GPT request... ERROR")
        return False
    
    with open('./batch/'+request_id+'.json', 'w') as f:
        f.write(batch_data)
    
    print("GPT request... Done")

    end_time = time.time()

    # # from pprint import pprint
    # # pprint(response)
    # json_output_path = current_path + r"/single_request_json_output"


    # try:
    #     os.makedirs(json_output_path)
    # except:
    #     pass

    # student_json_output_path = json_output_path + "/"
    # # + student + "/sections" 
    # try:
    #     os.makedirs(student_json_output_path)
    # except:
    #     pass

    # # TODO: choose better name
    # #  currently using the original image file name
    # image_json_output_path = student_json_output_path #+ "/" + image_name
    # try:
    #     os.makedirs(image_json_output_path)
    # except:
    #     pass

    # # copy image here
    # # copy_image(input_image_path, image_json_output_path)
        

    # json_output_path = image_json_output_path + "/" + gpt_model + "_result.json"

    # reponse_json = get_response_json(response, gpt_model, start_time, end_time)

    # with open(json_output_path, "w") as outfile:
    #     outfile.write(reponse_json)
        
    # print("Written to file "+json_output_path)

def scan_section_data(image_path):


    base64_image = encode_image(image_path)

    messages = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": """Je krijgt een foto van een Nederlandse toetsantwoord. 
                                Je moet deze omzetten in tekst. 
                                Deze toets moet nog worden nagekeken je. 
                                Verander niets aan de inhoud. 
                                Bedenk geen nieuwe woorden of woordonderdelen. 
                                Verander alleen kleine spelfoutjes.
                                houd in het antwoord ook rekening met meerdere regels en geeft die aan met een '\\n'
                                """
                                # de vraagnummers moeten getallen zijn
                                # als een vraagnummer een letter heeft, bijvoorbeeld 1a of 2c
                                # noteer dat dan al volgt: 1.a en 2.c dus, {getal}.{letter}
                },
                {
                    "type": "text",
                    "text": "Geef antwoord in JSON zoals in een aangegeven schema staat"    
                },
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "extraheer de tekst"    
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{base64_image}"
                    }
                }
            ]
        }
    ]
    


    num_tokens = num_tokens_from_messages(messages= messages,model="gpt-4o")

    # if (input(f"execute {num_tokens}? (y/n)") != "y" ):
        # exit()

    return messages



# scan_section(student, image_name)


In [116]:
import os
from PIL import Image
import sys
import psutil

def ask_image(path) -> bool:
    img = Image.open(path)
    img.show()
    result = input("Send to gpt (y/n) 'quit' = quit: ")
    return result
    
image_paths_to_process = []
    
i = 0    
if os.path.exists("output/"):
    
    for name in os.listdir("output/"):


        for question in os.listdir(f"output/{name}/questions/"):
            
            # scan = ask_image(f"section_output/{student}/sections/{section}")
            # control over usage of api
            # if scan == "y":
            clean_section_name = question.replace('.png', '' )
            
            i+=1
            print(f"Starting {i} - {name} - {question} ...")   
            path = f"./output/{name}/questions/{question}/{name}_q{question}.png"
            
            data = scan_section_data(path)
            single_request(data, name, question)
            image_paths_to_process.append(path)
            print(f"Starting {i} - {name} - {question} ... Done")   

            # if (scan == "quit"):
                # raise SystemExit(0)

scan_section_data = [scan_section_data(path) for path in image_paths_to_process]


# with open('./batch_requests/'+uuid.uuidv4()) as f:

# batch_request()



Starting 1 - input - 1 ...
GPT request... 
GPT request... Done
Written to file ./output/input/questions/1/json//gpt-4o-mini.json
Starting 1 - input - 1 ... Done
Starting 2 - input - 1n ...
GPT request... 
GPT request... Done
Written to file ./output/input/questions/1n/json//gpt-4o-mini.json
Starting 2 - input - 1n ... Done
Starting 3 - input - 4 ...
GPT request... 
GPT request... Done
Written to file ./output/input/questions/4/json//gpt-4o-mini.json
Starting 3 - input - 4 ... Done
